<a href="https://colab.research.google.com/github/chefPony/nn_zero_to_hero/blob/master/gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building a Transformer from scratch

## Load Data

In [1]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-09-07 16:43:25--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.3’

input.txt.3         100%[===================>]   1.06M  --.-KB/s    in 0.04s   

2024-09-07 16:43:25 (27.4 MB/s) - ‘input.txt.3’ saved [1115394/1115394]



In [2]:
with open("input.txt", "r", encoding="utf-8") as f:
    text = f.read()

In [3]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [4]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = {c: i for i, c in enumerate(chars)}
itos = {i: c for c, i in stoi.items()}

encode = lambda x: [stoi[c] for c in x]
decode = lambda x: [itos[i] for i in x]
print(vocab_size)
print("".join(chars))
print(itos)
print(stoi)

65

 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
{0: '\n', 1: ' ', 2: '!', 3: '$', 4: '&', 5: "'", 6: ',', 7: '-', 8: '.', 9: '3', 10: ':', 11: ';', 12: '?', 13: 'A', 14: 'B', 15: 'C', 16: 'D', 17: 'E', 18: 'F', 19: 'G', 20: 'H', 21: 'I', 22: 'J', 23: 'K', 24: 'L', 25: 'M', 26: 'N', 27: 'O', 28: 'P', 29: 'Q', 30: 'R', 31: 'S', 32: 'T', 33: 'U', 34: 'V', 35: 'W', 36: 'X', 37: 'Y', 38: 'Z', 39: 'a', 40: 'b', 41: 'c', 42: 'd', 43: 'e', 44: 'f', 45: 'g', 46: 'h', 47: 'i', 48: 'j', 49: 'k', 50: 'l', 51: 'm', 52: 'n', 53: 'o', 54: 'p', 55: 'q', 56: 'r', 57: 's', 58: 't', 59: 'u', 60: 'v', 61: 'w', 62: 'x', 63: 'y', 64: 'z'}
{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b

In [5]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

encoded_data = torch.tensor([stoi[c] for c in text])
print(encoded_data[:1000])

train_data = encoded_data[:int(len(text) * 0.9)]
val_data = encoded_data[int(len(text) * 0.9):]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
        47, 59, 57,  1, 47, 57,  1, 41, 

In [6]:
torch.manual_seed(42)

def get_batch(data: torch.Tensor, batch_size: int, block_size: int):
  ix = torch.randint(low=0, high=data.shape[0] - block_size, size=(batch_size, 1))
  xb = torch.stack([data[i : i+block_size] for i in ix], dim=0).to(device)
  yb = torch.stack([data[i+1 : i+block_size+1] for i in ix], dim=0).to(device)
  return xb, yb

xb, yb = get_batch(encoded_data, 4, 8)
print(xb.shape)
print(yb.shape)
print("batch")
print(xb)
print("target")
print(yb)

torch.Size([4, 8])
torch.Size([4, 8])
batch
tensor([[42,  1, 58, 46, 59, 57,  1, 21],
        [54, 56, 47, 43, 57, 58, 11,  0],
        [49, 47, 52, 45, 12,  1, 58, 46],
        [58, 46, 53, 59, 58,  1, 56, 43]], device='cuda:0')
target
tensor([[ 1, 58, 46, 59, 57,  1, 21,  1],
        [56, 47, 43, 57, 58, 11,  0, 37],
        [47, 52, 45, 12,  1, 58, 46, 53],
        [46, 53, 59, 58,  1, 56, 43, 42]], device='cuda:0')


## Baseline: Bigram Model

In [7]:
import torch.nn as nn
from torch.nn import functional as F

class BigramModel(nn.Module):

  def __init__(self, vocab_size: int):
    super().__init__()
    self.logits = nn.Embedding(num_embeddings=vocab_size, embedding_dim=vocab_size)


  def forward(self, x, targets=None):
    logits = self.logits(x) # (B, T, C)
    if targets is None:
      loss = None
    else:
       B, T, C = logits.shape
       loss = F.cross_entropy(logits.view(B * T, C), targets.view(B * T))
    return logits, loss

  @torch.no_grad
  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):
      logits, _ = self(idx)  #(B, n_vocab)
      probs = F.softmax(logits, dim=-1)[:, -1, :]
      next_idx = torch.multinomial(probs, 1)
      idx = torch.cat([idx, next_idx], dim=1)
    return idx


xb, yb = get_batch(train_data, batch_size=8, block_size=2)
bigram = BigramModel(vocab_size).to(device)
print(bigram(xb, yb))
print("".join(decode(bigram.generate(torch.tensor([[0 , 0]], device=device), 100).squeeze().tolist())))

(tensor([[[-0.1827,  0.0524, -1.8020,  ..., -0.4538,  0.6346, -1.4856],
         [-1.1441,  0.3383,  1.6992,  ...,  0.9254,  1.4805,  0.3449]],

        [[-0.1827,  0.0524, -1.8020,  ..., -0.4538,  0.6346, -1.4856],
         [-1.2800,  0.1359, -1.2744,  ...,  1.1272,  0.5445, -0.2186]],

        [[-1.0800,  1.4510, -0.3488,  ...,  2.1158,  0.2643, -0.2391],
         [ 0.4121, -1.9089, -0.0616,  ..., -0.6875,  0.2056, -0.7192]],

        ...,

        [[-0.4899, -1.5937,  0.9481,  ...,  0.8930,  1.6673, -1.0136],
         [ 2.5165, -0.0862,  0.1101,  ..., -0.6886, -0.2301,  0.0784]],

        [[ 1.0688,  1.0354, -1.0889,  ..., -0.9309, -0.7496, -1.1346],
         [-1.1441,  0.3383,  1.6992,  ...,  0.9254,  1.4805,  0.3449]],

        [[ 0.4310, -0.2231,  0.2790,  ..., -0.3801, -0.2620, -0.5226],
         [-1.0800,  1.4510, -0.3488,  ...,  2.1158,  0.2643, -0.2391]]],
       device='cuda:0', grad_fn=<EmbeddingBackward0>), tensor(4.6520, device='cuda:0', grad_fn=<NllLossBackward0>))


n
3

In [ ]:
lr = 1.
block_size = 2
batch_size = 16
n_iter = 20000
model = BigramModel(vocab_size).to(device)

optimizer = torch.optim.Adam(params=model.parameters(), lr=1.)

@torch.no_grad
def evaluate_model(model, batch_size, block_size, num_batches=100):
  loss_tr, loss_va = 0, 0
  for _ in range(num_batches):
    xb_tr, yb_tr = get_batch(train_data, batch_size, block_size)
    xb_va, yb_va = get_batch(val_data, batch_size, block_size)
    _, lossb_tr = model(xb_tr, yb_tr)
    _, lossb_va = model(xb_va, yb_va)
    loss_tr += lossb_tr
    loss_va += lossb_va
  return loss_tr/num_batches, loss_va/num_batches

for i in range(n_iter):
  xb, yb = get_batch(train_data, batch_size, 2)
  _, loss = model(xb, yb)

  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

  if i % 1000 == 0:
    loss_tr, loss_va = evaluate_model(model, batch_size, block_size)
    print(f"Train: {loss_tr:.4f} Validation: {loss_va:.4f}")

Train: 4.3754 Validation: 4.4458
Train: 3.7335 Validation: 3.9622
Train: 3.7469 Validation: 3.9400
Train: 3.6187 Validation: 3.6909
Train: 3.8301 Validation: 3.8622
Train: 3.8224 Validation: 3.9138
Train: 3.6752 Validation: 3.9924
Train: 3.8573 Validation: 3.9302
Train: 3.7383 Validation: 3.7476
Train: 3.7376 Validation: 3.9901
Train: 3.9125 Validation: 4.0334


In [ ]:
print("".join(decode(model.generate(torch.tensor([[1 , 1]], device=device), 100).squeeze().tolist())))

## Transformer model

In [ ]:
import math

class HeadAttention(nn.Module):

  def __init__(self, input_embd, head_size, vdim=None, dropout=0):
    super().__init__()
    self.input_embd = input_embd
    self.head_size = head_size
    self.vdim = vdim if vdim else head_size
    self.K = nn.Linear(self.input_embd, self.head_size, bias=False)
    self.Q = nn.Linear(self.input_embd, self.head_size, bias=False)
    self.V = nn.Linear(self.input_embd, self.vdim, bias=False)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x, y=None):
    B, T, C = x.shape
    # x[i, j, :] stores the char and positional info for token[i, j]
    # K, Q layers use that information to compute the query and key for each token
    k, q = self.K(x), self.Q(x) # (B, T, H), (B, T, H)
    v = self.V(x) # (B, T, v)
    x = q @ k.transpose(1, 2) * (self.head_size**-0.5) # (B, T, T)
    # mask to disable flow of information from future tokens
    mask = torch.tril(torch.ones((T, T)) == 0).to(device)
    x = x.masked_fill_(mask, -math.inf)
    x = F.softmax(x, dim=-1)
    x = self.dropout(x)
    x = x @ v # (B, T, v)
    return x

class MultiHeadAttention(nn.Module):

  def __init__(self, input_embd, total_head_size, n_heads, dropout=0):
    super().__init__()
    self.input_embd = input_embd
    self.total_head_size = total_head_size
    self.n_heads = n_heads
    self.heads = nn.ModuleList([
        HeadAttention(self.input_embd, self.total_head_size//self.n_heads, dropout=dropout)
        for _ in range(n_heads)])
    self.proj = nn.Linear(self.total_head_size, self.total_head_size//self.n_heads)

  def forward(self, x):
    x = torch.cat([h(x) for h in self.heads], dim=-1)
    x = self.proj(x)
    return x

class LayerNorm(nn.Module):

  def __init__(self, dim, eps=1e-8):
    super().__init__()
    self.eps = eps
    self.gamma = torch.ones(dim, device=device)
    self.beta = torch.zeros(dim, device=device)

  def forward(self, x):
    B, T, C = x.shape
    mu = torch.mean(x, dim=(1, 2), keepdim=True) # (B, 1, 1)
    sigma = torch.std(x, dim=(1, 2), keepdim=True) # (B, 1, 1)
    # (B, T, C) (B)
    x = (x - mu) * (sigma + self.eps)**-0.5 * self.gamma + self.beta
    return x

class FeedForward(nn.Module):

  def __init__(self, input_dim, hidden_dim, output_dim, activation, dropout=0):
    super().__init__()
    self.input_dim = input_dim
    self.hidden_dim = hidden_dim
    self.output_dim = output_dim
    self.activation = activation
    self.net = nn.Sequential(*[
        nn.Linear(self.input_dim, self.hidden_dim),
        activation,
        nn.Linear(self.hidden_dim, self.output_dim),
        nn.Dropout(dropout)
    ])

  def forward(self, x):
    return self.net(x)

class Block(nn.Module):

  def __init__(self, input_embd, total_head_size, n_heads, dropout=0):
    super().__init__()
    hidden_dim = total_head_size // n_heads
    self.ma = MultiHeadAttention(input_embd, total_head_size, n_heads, dropout)
    self.fa = FeedForward(hidden_dim, hidden_dim, hidden_dim, nn.GELU(), dropout)
    self.ln1 = LayerNorm(hidden_dim)
    self.ln2 = LayerNorm(hidden_dim)

  def forward(self, x):
    # skip connections
    x = x + self.ma(self.ln1(x))
    x = x + self.fa(self.ln2(x))
    return x

class Transformer(nn.Module):

  def __init__(self, block_size, embd, n_blocks, n_heads, dropout=0):
    super().__init__()
    self.C = nn.Embedding(vocab_size, embd)
    self.P = nn.Embedding(block_size, embd)
    self.blocks = nn.Sequential(*[
        Block(embd, embd * n_heads, n_heads, dropout)
        for _ in range(n_blocks)])
    self.logits = nn.Linear(embd, vocab_size)

  def forward(self, x, y=None):
    B, T = x.shape
    p = torch.ones((B, T), device=device) * torch.arange(0, T, device=device)
    p = self.P(p.long())
    x = self.C(x) + p
    x = self.blocks(x)
    x = self.logits(x)
    if y is None:
      loss = None
    else:
      loss = F.cross_entropy(x.view(B*T, vocab_size), y.view(B*T))
    return x, loss

  @torch.no_grad
  def generate(self, idx, max_new_tokens=100):
    for _ in range(max_new_tokens):
      logits, _ = self(idx)  #(B, n_vocab)
      probs = F.softmax(logits, dim=-1)[:, -1, :]
      next_idx = torch.multinomial(probs, 1)
      idx = torch.cat([idx, next_idx], dim=1)
    return idx

x = torch.randint(0, 65, size=(4, 8), device=device)
l = Transformer(8, 16, 2, 2).to(device)
l(x)[0].shape

In [ ]:
out = model.generate(torch.ones((2, block_size), device=device).long(), 100)

In [11]:
n_iter = 1000
batch_size = 32
block_size = 128
embd = 32
n_blocks = 3
n_heads = 4
dropout = 0.1

@torch.no_grad
def evaluate_model(model, batch_size, block_size, num_batches=100):
  loss_tr, loss_va = 0, 0
  for _ in range(num_batches):
    xb_tr, yb_tr = get_batch(train_data, batch_size, block_size)
    xb_va, yb_va = get_batch(val_data, batch_size, block_size)
    yhtr, lossb_tr = model(xb_tr, yb_tr)
    yhva, lossb_va = model(xb_va, yb_va)
    loss_tr += lossb_tr
    loss_va += lossb_va
  return loss_tr/num_batches, loss_va/num_batches


print(device)
model = Transformer(block_size, embd, n_blocks, n_heads, dropout).to(device)
print(f"Parameters {sum(p.numel() for p in model.parameters())}")
model.to(device)

optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-3)

for i in range(n_iter):
  xb, yb = get_batch(train_data, batch_size, block_size)
  yhat, loss = model(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

  if i % 100 == 0:
    loss_tr, loss_va = evaluate_model(model, batch_size, block_size)
    print(f"Step {i}/{n_iter} Train: {loss_tr:.4f} Validation: {loss_va:.4f}")

cuda
Parameters 63905
Step 0/1000 Train: 4.5743 Validation: 4.5765
Step 100/1000 Train: 2.8868 Validation: 2.9119
Step 200/1000 Train: 2.6765 Validation: 2.6849
Step 300/1000 Train: 2.5918 Validation: 2.6036
Step 400/1000 Train: 2.5446 Validation: 2.5568
Step 500/1000 Train: 2.5144 Validation: 2.5257
Step 600/1000 Train: 2.4923 Validation: 2.5032
Step 700/1000 Train: 2.4642 Validation: 2.4820
Step 800/1000 Train: 2.4445 Validation: 2.4611
Step 900/1000 Train: 2.3993 Validation: 2.4181


In [14]:
out = model.generate(torch.ones((2, block_size), device=device).long(), 100)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [116]:
yhat.shape

torch.Size([32, 128, 65])